# OpenAI Finetuning REACT- Distill GPT-4 to GPT-3.5

In this notebook, we walk through an example of fine-tuning gpt-3.5-turbo.

Specifically, we attempt to distill GPT-4's knowledge, by generating training data with GPT-4 to then fine-tune GPT-3.5.

All training data is generated using two different sections of our index data, creating both a training and evalution set.

We then finetune with our `OpenAIFinetuneEngine` wrapper abstraction.

Evaluation is done using the `ragas` library, which we will detail later on.

In [6]:
%pip install llama-index-finetuning
%pip install llama-index-finetuning-callbacks
%pip install llama-index-llms-openai

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:001.4 MB/s eta 0:00:01
  Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (150 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 7.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 6.6 MB/s eta 0:00:000m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 13.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6

In [12]:
!pip install pydantic  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 1.2 MB/s eta 0:00:001.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 kB 4.1 MB/s eta 0:00:003.1 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradientai 1.8.0 requires pydantic<2.0.0,>=1.10.5, but you have pydantic 2.6.3 which is incompatible.


In [1]:
import os
import openai
 

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import (
    OpenAIEmbedding,
)  # pants: no-infer-dep

from llama_index.core.tools import QueryEngineTool, ToolMetadata

## Data Setup

Here, we first down load the PDF that we will use to generate training data.

The next step is generating a training and eval dataset.

We will generate 40 questions on different sections of the PDF we downloaded.

We can use GPT-3.5 on the eval questions to get our baseline performance.

Then, we will use GPT-4 on the train questions to generate our training data. The training data will be collected with out `OpenAIFineTuningHandler`.

This step is entirely optional if you don't want to spend the time/tokens -- the eval and training questions are also provided in this folder, as well as the training data!

### Train Generation

In [8]:
 
llm = OpenAI(model="gpt-4", temperature=0.3, api_key = "sk-ApUK41y73g8qMbrz36A81641752946449f10BbBe32Ff2b7c",api_base="http://localhost:3000/v1")
embeddings = OpenAIEmbedding(api_key = "EMPTY",api_base="http://127.0.0.1:9997/v1")
# os.environ['OPENAI_API_KEY'] = ""

In [9]:

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/marchtest2"
    )
    march_index = load_index_from_storage(storage_context)
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/junetest2"
    )
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/septtest2"
    )
    sept_index = load_index_from_storage(storage_context)
    index_loaded = True
except:
    index_loaded = False
if not index_loaded:
 
     # load data
    march_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Documents/张毛峰个人简历 - 2024-01-20(1).pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Downloads/个人简历_刘立兼(1).docx"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["/home/dmeck/Downloads/简历_宋金珂_北京交通大学_网络空间安全.pdf"]
    ).load_data()

    # build index
    march_index = VectorStoreIndex.from_documents(
        march_docs,embed_model=embeddings
    )
    june_index = VectorStoreIndex.from_documents(
        june_docs,embed_model=embeddings
    )
    sept_index = VectorStoreIndex.from_documents(
        sept_docs,embed_model=embeddings
    )
 
    # persist index
    march_index.storage_context.persist(persist_dir="./storage/marchtest2")
     
    june_index.storage_context.persist(persist_dir="./storage/junetest2")
    sept_index.storage_context.persist(persist_dir="./storage/septtest2")

In [10]:
march_engine = march_index.as_query_engine(similarity_top_k=3, llm=llm)
june_engine = june_index.as_query_engine(similarity_top_k=3, llm=llm)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, llm=llm)

In [11]:
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=(
        f"关于张毛峰的简历信息，包括了langchain-chatchat、InterpretationoDreams、KM 平台、省检修特高压生产指挥管控系统、智能运检移动应用、福建监控系统项目"

    ),
)

query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=(
        f"关于刘立兼的简历信息，包括了•篝火心理、雷鸟365、雷鸟365、网聚宝CRM、AP数据基盘、AP数据基盘等项目"
    ),
)
query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=(
        f"关于宋金珂的简历信息，包括了全球 IPv4 空间内的物联网设备扫描识别和隐私安全分、开源软件生态内的跨项目依赖分析及漏洞影响追溯、已发表论文列表、IoT 设备安全等项目"
    ),
)

query_engine_tools = [query_tool_march, query_tool_june, query_tool_sept]

In [12]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [13]:
 
base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [18]:
# gpt-3.5 generally gives the right response here
response = base_agent.chat(
    "宋金柯发表了什么论文,详细的是什么"
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: sept_2022
Action Input: {'input': '宋金珂发表了什么论文'}
Observation: [1] Jinke Song , Qiang Li, Haining  Wang , Limin  Sun. Under the conceali ng su rface: Detecting and understanding 
live webcams in the wild[J]. Proceedings of the ACM on Measu rement and Analysis of Computing Systems  
(SIGMETRICS ), 2020, 4(1): 1 -25. 
[2] Qiang Li, Jinke Song , Dawei  Tan, Haining Wang, Jiqiang Liu . PDGraph : A Large -Scale Empirica sl Study on 
Project Dependency o f Security  Vulnerabilitie s[C]. 51st Annual IEEE/IFIP Int ernational Conference on Dependable 
Systems and Networks ( DSN ). IEEE, 2021: 161 -173. 
[3] Zhihao  Wang , Qiang Li, Jinke Song , Haining Wang, L imin Sun. Towards IP -based geolocation via fine -grained 
and s table webcam landmarks[C] . Proceedings of The Web Confer ence (WWW ). 2020: 1422 -1432.  
[4] Li, Q., Wang, Z., Tan, D., Jinke Song , Wang, H. etc. (2021). GeoCAM: An IP -Based Geolocation Service 
Through 